# SDP relaxations for the CHSH game

In [1]:
import lib_non_local_games as nlg
import numpy as np
import cvxpy as cp
import functools as fc
from operator import mul

In [2]:
cp.installed_solvers()

['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'MOSEK', 'OSQP', 'SCS']

Parameter of the CHSH game

In [3]:
dimA1 = 2
dimA2 = 2
dimQ1 = 2
dimQ2 = 2

dimT = 2
dimS = 2

probQ1 = (.5,.5)
probQ2 = (.5,.5)

# CHSH game $n_1 = 1$, $n_2 = 1$, and $|T| = 2$

### With CQ assumption and complex variables

We consider the variable

$\rho_{A_1 Q_1 A_2 Q_2 T \hat{T} S \hat{S}} = \sum_{a_1,q_1,a_2,q_2} |a_1,q_1,a_2,q_2 \rangle \langle a_1,q_1,a_2,q_2|_{A_1 Q_1 A_2 Q_2} \otimes \tilde{\rho}(a_1,q_1,a_2,q_2)_{T \hat{T} S \hat{S}}$,

where the states $ \left\{\tilde{\rho}(a_1,q_1,a_2,q_2)\right\}$ are the variables, and they are sub-normalized so that $\mathrm{Tr} \left[ \tilde{\rho}(a_1,q_1,a_2,q_2) \right] = p(a_1,q_1,a_2,q_2)$.

In [4]:
# Subsystems A1 Q1 A2 Q2
subs_A1Q1A2Q2 = (dimA1,dimQ1,dimA2,dimQ2)
indices_A1Q1A2Q2 = nlg.indices_list(subs_A1Q1A2Q2)
dim_A1Q1A2Q2 = fc.reduce(mul, subs_A1Q1A2Q2, 1)

# Subsystems A2 Q1 Q2
subs_A2Q1Q2 = (dimA2,dimQ1,dimQ2)
indices_A2Q1Q2 = nlg.indices_list(subs_A2Q1Q2)

# Subsystems A1 Q1 Q2
subs_A1Q1Q2 = (dimA1,dimQ1,dimQ2)
indices_A1Q1Q2 = nlg.indices_list(subs_A1Q1Q2)

# Subsystems A1 Q1
subs_A1Q1 = (dimA1,dimQ1)
indices_A1Q1 = nlg.indices_list(subs_A1Q1)

# Subsystems A2 Q2
subs_A2Q2 = (dimA2,dimQ2)
indices_A2Q2 = nlg.indices_list(subs_A2Q2)

# Subsystems T hat(T) S hat(S)
subs_TTSS = (dimT,dimT,dimS,dimS)
indices_TTSS = nlg.indices_list(subs_TTSS)
dim_TTSS = fc.reduce(mul, subs_TTSS, 1)

# Additional dimensions
dim_TSS = dimT * dimS**2
dim_TT = dimT**2
dim_SS = dimS**2

# State on subsystem T
rhoT = np.identity(dimT)/dimT

The program

In [5]:
## VARIABLES

# The (sub-normalized) states we optimize over
rho_TTSS = []
for i in map(nlg.binarytoint,indices_A1Q1A2Q2):
    rho_TTSS.append( cp.Variable((dim_TTSS,dim_TTSS),hermitian=True) )

## OBJECTIVE FUNCTION

# The rule function is
rule_A1Q1A2Q2 = nlg.CHSH_rule_function_A1Q1A2Q2

# The swap operator is
F_TTSS = nlg.permutation_matrix((0,1), (1,0), (dim_TT, dim_SS))

# The object function is
obj_fun_components = [rule_A1Q1A2Q2(*index) * cp.trace( cp.matmul(F_TTSS,rho_TTSS[nlg.binarytoint(index)]) )
                      for index in indices_A1Q1A2Q2]

object_function = cp.Constant(dimT**2) * sum(obj_fun_components)

## CONSTRAINTS

constraints = []
    
# 2) rho_TTSS are (sub-normalized) quantum states
# 2a) trace of the sum is 1
constraints.append( sum([cp.trace(rho_TTSS[i]) for i in map(nlg.binarytoint,indices_A1Q1A2Q2)]) - 1 == 0 )

# 2b) positive semidefinite matrices
for i in map(nlg.binarytoint,indices_A1Q1A2Q2):
    constraints.append( rho_TTSS[i] >> 0 )
    
# 3) First linear constraint
for a2,q1,q2 in indices_A2Q1Q2:
    indices_A1q1a2q2 = [nlg.binarytoint([a,q1,a2,q2]) for a in range(dimA1)]
    indices_A1Q1a2q2 = [nlg.binarytoint([a,q,a2,q2]) for a,q in indices_A1Q1]
    
    lhs = sum([rho_TTSS[i] for i in indices_A1q1a2q2])
    
    rhs_variable = sum([rho_TTSS[i] for i in indices_A1Q1a2q2])
    rhs_partial = nlg.partial_trace(rhs_variable, [dimT, dim_TSS])
    rhs = cp.Constant(probQ1[q1]) * cp.kron(rhoT, rhs_partial)
    
    constraints.append( lhs - rhs == 0 )
    
# 4) Second linear constraint
P = nlg.permutation_matrix((0,1,2,3),(1,0,2,3),subs_TTSS)

for a1,q1,q2 in indices_A1Q1Q2:
    indices_a1q1A2q2 = [nlg.binarytoint([a1,q1,a,q2]) for a in range(dimA2)]
    indices_a1q1A2Q2 = [nlg.binarytoint([a1,q1,a,q]) for a,q in indices_A2Q2]
    
    lhs_variable = sum([rho_TTSS[i] for i in indices_a1q1A2q2])
    lhs = cp.matmul( cp.matmul(P,lhs_variable) , P )
    
    rhs_variable = sum([rho_TTSS[i] for i in indices_a1q1A2Q2])
    rhs_permuted = cp.matmul( cp.matmul(P,rhs_variable) , P )
    rhs_partial = nlg.partial_trace(rhs_permuted, [dimT, dim_TSS])
    rhs = cp.Constant(probQ2[q2]) * cp.kron(rhoT, rhs_partial)
    
    constraints.append( lhs - rhs == 0 )
    
# 5) PPT criterium
for i in map(nlg.binarytoint,indices_A1Q1A2Q2):
    constraints.append( nlg.partial_transpose(rho_TTSS[i],subs_TTSS,(0,0,1,1)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_TTSS[i],subs_TTSS,(1,0,0,0)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_TTSS[i],subs_TTSS,(0,1,0,0)) >> 0 )
    
# Write the problem
prob = cp.Problem(cp.Maximize(cp.real(object_function)), constraints)

Solve the problem

In [6]:
prob.solve(verbose=True,solver='MOSEK')



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 73730           
  Cones                  : 0               
  Scalar variables       : 6272            
  Matrix variables       : 64              
  Integer variables      : 0               

Optimizer started.
GP based matrix reordering started.
GP based matrix reordering terminated.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 73730           
  Cones                  : 0               
  Scalar variables       : 6272            
  Matrix variables       : 64              
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            

1.000000016296899

# CHSH game $n_1 = 2$, $n_2 = 1$, and $|T| = 2$

### With CQ assumptions  and real variables

This is now a variable

$\rho_{(A^{(1)}_1 Q^{(1)}_1 T^{(1)}) (A^{(2)}_1 Q^{(2)}_1 T^{(2)}) (A_2 Q_2 \hat{T}) (S \hat{S})} = \sum_{a_1,a'_1,a_2,q_1,q'_1,q_2} |a_1,a'_1,q_1,q'_1,a_2,q_2 \rangle \langle a_1,a'_1,q_1,q'_1,a_2,q_2|_{A^{(1)}_1 A^{(2)}_1 Q^{(1)}_1 Q^{(2)}_1 A_2 Q_2} \otimes \tilde{\rho}(a_1,a'_1,q_1,q'_1,a_2,q_2)_{T^{(1)} T^{(2)} \hat{T} S \hat{S}}$,

where the states $ \left\{\tilde{\rho}(a_1,a'_1,q_1,q'_1,a_2,q_2)\right\}$ are the variables, and they are sub-normalized so that $\mathrm{Tr} \left[ \tilde{\rho}(a_1,a'_1,q_1,q'_1,a_2,q_2) \right] = p(a_1,a'_1,q_1,q'_1,a_2,q_2)$.

In [4]:
# Subsystems A(1)1 A(2)1 Q(1)1 Q(2)1 A2 Q2
subs_AAQQ1_AQ2 = (dimA1,dimA1,dimQ1,dimQ1,dimA2,dimQ2)
indices_AAQQ1_AQ2 = nlg.indices_list(subs_AAQQ1_AQ2)

# Subsystems T(1)1 T(2)1 T2 S1 S2
subs_TT1_T2_S1_S2 = (dimT,dimT,dimT,dimS,dimS)
dim_TT1_T2_S1_S2 = fc.reduce(mul, subs_TT1_T2_S1_S2, 1)

# Subsystems T(*)1 T2 S1 S2
subs_T1_T2_S1_S2 = (dimT,dimT,dimS,dimS)
dim_T1_T2_S1_S2 = fc.reduce(mul, subs_T1_T2_S1_S2, 1)

# Subsystems A(1)1 Q(1)1 A2 Q2
subs_A1Q1A2Q2 = (dimA1,dimQ1,dimA2,dimQ2)
indices_A1Q1A2Q2 = nlg.indices_list(subs_A1Q1A2Q2)

# Subsystems A(2)1 Q(1)1 Q(2)1 A2 Q2
subs_AQQ1_AQ2 = (dimA1,dimQ1,dimQ1,dimA2,dimQ2)
indices_AQQ1_AQ2 = nlg.indices_list(subs_AQQ1_AQ2)

# Subsystems A(1)1 A(2)1 Q(1)1 Q(2)1 Q2
subs_AAQQ1_Q2 = (dimA1,dimA1,dimQ1,dimQ1,dimQ2)
indices_AAQQ1_Q2 = nlg.indices_list(subs_AAQQ1_Q2)

# Subsystems A1 Q1
subs_A1Q1 = (dimA1,dimQ1)
indices_A1Q1 = nlg.indices_list(subs_A1Q1)

# Subsystems A2 Q2
subs_A2Q2 = (dimA2,dimQ2)
indices_A2Q2 = nlg.indices_list(subs_A2Q2)

# State on subsystem T
rhoT = np.identity(dimT)/dimT

The program

In [5]:
## VARIABLES 

# The (sub-normalized) states we optimize over
rho_T1T2TSS = []
for i in map(nlg.binarytoint,indices_AAQQ1_AQ2):
    rho_T1T2TSS.append( cp.Variable((dim_TT1_T2_S1_S2,dim_TT1_T2_S1_S2),symmetric=True) )

## OBJECTIVE FUNCTION

# The swap operator takes T(1)1 T(2)1 T2 S1 S2 to S1 T(2)1 S2 T(1)1 T2
F_TTTSS = cp.Constant(nlg.permutation_matrix((0,1,2,3,4), (3,1,4,0,2), subs_TT1_T2_S1_S2))

# The object function is
obj_fun_components = []
for a1,q1,a2,q2 in indices_A1Q1A2Q2:
    v_a1q1a2q2 = cp.Constant( int(nlg.CHSH_rule_function_A1Q1A2Q2(a1,q1,a2,q2)) )
    variable_T1T2TSS = sum([rho_T1T2TSS[nlg.binarytoint([a1,a,q1,q,a2,q2])] for a,q in indices_A1Q1])
    obj_fun_components.append( v_a1q1a2q2 * cp.trace( cp.matmul(F_TTTSS,variable_T1T2TSS) ) )
    
object_function = cp.Constant(dimT**2) * sum(obj_fun_components)

## CONSTRAINTS

constraints = []

# 1) rho_T1T2TSS are (sub-normalized) quantum states
# 1a) trace of the sum is 1
constraints.append( sum([cp.trace(rho_T1T2TSS[i]) for i in map(nlg.binarytoint,indices_AAQQ1_AQ2)]) - 1 == 0 )

# 1b) positive semidefinite matrices
for i in map(nlg.binarytoint,indices_AAQQ1_AQ2):
    constraints.append( rho_T1T2TSS[i] >> 0 )
    
# 2) Permutation invariance over A(1)1 Q(1)1 T1 and A(2)1 Q(2)1 T2

# The permutation matrix swaps T(1)1 and T(2)1
P = cp.Constant(nlg.permutation_matrix((0,1,2,3,4), (1,0,2,3,4), subs_TT1_T2_S1_S2))

for a1,a,q1,q,a2,q2 in indices_AAQQ1_AQ2:
    lhs = rho_T1T2TSS[nlg.binarytoint([a1,a,q1,q,a2,q2])]
    
    rhs_variable = rho_T1T2TSS[nlg.binarytoint([a,a1,q,q1,a2,q2])]
    rhs = cp.matmul(cp.matmul(P,rhs_variable),P)
    
    constraints.append( lhs - rhs == 0 )

# 3) First linear constraint
for a,q1,q,a2,q2 in indices_AQQ1_AQ2:
    indices_A1aq1qa2q2 = [nlg.binarytoint([a1,a,q1,q,a2,q2]) for a1 in range(dimA1)]
    indices_A1aQ1qa2q2 = [nlg.binarytoint([a1,a,qq1,q,a2,q2]) for a1,qq1 in indices_A1Q1]
    
    lhs = sum([rho_T1T2TSS[i] for i in indices_A1aq1qa2q2])
    
    rhs_variable = sum([rho_T1T2TSS[i] for i in indices_A1aQ1qa2q2])
    rhs_partial = nlg.partial_trace(rhs_variable, [dimT, dim_T1_T2_S1_S2])
    rhs = cp.Constant(probQ1[q1]) * cp.kron(cp.Constant(rhoT), rhs_partial)
    
    constraints.append( lhs - rhs == 0 )
    
# 4) Second linear constraint

# The permutation matrix swaps T(1)1 and T2
P = cp.Constant(nlg.permutation_matrix((0,1,2,3,4), (2,1,0,3,4), subs_TT1_T2_S1_S2))

for a1,a,q1,q,q2 in indices_AAQQ1_Q2:
    indices_a1aq1qA2q2 = [nlg.binarytoint([a1,a,q1,q,a2,q2]) for a2 in range(dimA2)]
    indices_a1aq1qA2Q2 = [nlg.binarytoint([a1,a,q1,q,a2,qq2]) for a2,qq2 in indices_A2Q2]
    
    lhs_variable = sum([rho_T1T2TSS[i] for i in indices_a1aq1qA2q2])
    lhs = cp.matmul( cp.matmul(P,lhs_variable) , P )
    
    rhs_variable = sum([rho_T1T2TSS[i] for i in indices_a1aq1qA2Q2])
    rhs_permuted = cp.matmul( cp.matmul(P,rhs_variable) , P )
    rhs_partial = nlg.partial_trace(rhs_permuted, [dimT, dim_T1_T2_S1_S2])
    rhs = cp.Constant(probQ2[q2]) * cp.kron(cp.Constant(rhoT), rhs_partial)
    
    constraints.append( lhs - rhs == 0 )
    
# 5) PPT criterium
for i in map(nlg.binarytoint,indices_AAQQ1_AQ2):
    constraints.append( nlg.partial_transpose(rho_T1T2TSS[i],subs_TT1_T2_S1_S2,(0,0,0,1,1)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_T1T2TSS[i],subs_TT1_T2_S1_S2,(1,1,0,0,0)) >> 0 )
    constraints.append( nlg.partial_transpose(rho_T1T2TSS[i],subs_TT1_T2_S1_S2,(0,0,1,0,0)) >> 0 )
    
# Write the problem
prob = cp.Problem(cp.Maximize(object_function), constraints)

In [6]:
prob.solve(verbose=True,solver='MOSEK')



Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 393217          
  Cones                  : 0               
  Scalar variables       : 33792           
  Matrix variables       : 256             
  Integer variables      : 0               

Optimizer started.
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 393217          
  Cones                  : 0               
  Scalar variables       : 33792           
  Matrix variables       : 256             
  Integer variables      : 0               

Optimizer  - threads                : 2               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 266860
Optimizer  - Cones                  : 1
Optimizer  - Scalar variab

0.8750001385396399